[모델 ANN]
1. 3개의 은닉층을 사용하는 ANN 모델구조
- 정규분포 (tf.random_normal), ReLU 활성화 함수 사용
2. 손실함수 : MSE
3. 옵티마이저 : Adam Optimizer (learning rate=0.005)
4. 학습 epoch=400
- batch size=16

In [64]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [65]:
# 데이터 로드 함수
def load_csv(file_path):
    data = np.genfromtxt(file_path, delimiter=',', skip_header=1, usecols=range(8))
    x_data = np.delete(data, 1, axis=1)  # 두 번째 열 제거 (입력 변수)
    y_data = data[:, 1].reshape(-1, 1)  # 두 번째 열이 목표 변수
    return x_data, y_data

# 데이터 분할 및 표준화
def split_data(x_data, y_data):
    x_test = x_data[:400]
    y_test = y_data[:400]
    x_train = x_data[400:]
    y_train = y_data[400:]

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    return x_train, x_test, y_train, y_test

# 데이터 로드 및 분할
x_data, y_data = load_csv('data-ori.csv')
x_train, x_test, y_train, y_test = split_data(x_data, y_data)

In [66]:
# 하이퍼파라미터 설정
learning_rate = 0.005
num_epochs = 400
batch_size = 16
input_size = x_train.shape[1]
hidden1_size = 64
hidden2_size = 16
hidden3_size = 4
output_size = 1

In [67]:
# 입력과 출력 플레이스홀더 정의
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, output_size])

In [68]:
def build_ANN(x):
    W1 = tf.Variable(tf.random_normal([input_size, hidden1_size]))
    b1 = tf.Variable(tf.random_normal([hidden1_size]))
    H1_output = tf.nn.relu(tf.matmul(x, W1) + b1)

    W2 = tf.Variable(tf.random_normal([hidden1_size, hidden2_size]))
    b2 = tf.Variable(tf.random_normal([hidden2_size]))
    H2_output = tf.nn.relu(tf.matmul(H1_output, W2) + b2)

    W3 = tf.Variable(tf.random_normal([hidden2_size, hidden3_size]))
    b3 = tf.Variable(tf.random_normal([hidden3_size]))
    H3_output = tf.nn.relu(tf.matmul(H2_output, W3) + b3)

    W_output = tf.Variable(tf.random_normal([hidden3_size, output_size]))
    b_output = tf.Variable(tf.random_normal([output_size]))
    
    logits = tf.matmul(H3_output, W_output) + b_output
    return logits

# ANN 출력 값 예측
predicted_value = build_ANN(x)

In [69]:
# 손실 함수 (Mean Squared Error)
loss = tf.reduce_mean(tf.square(predicted_value - y))

# Adam 옵티마이저 선언
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

In [70]:
# 세션 및 학습 과정 정의
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 에포크 반복
    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = int(len(x_train) / batch_size)
        
        # 미니배치 학습
        for i in range(num_batches):
            batch_x = x_train[i * batch_size: (i + 1) * batch_size]
            batch_y = y_train[i * batch_size: (i + 1) * batch_size]
            
            _, batch_loss = sess.run([train_step, loss], feed_dict={x: batch_x, y: batch_y})
            total_loss += batch_loss
        
        # 검증 데이터로 성능 평가
        if epoch % 50 == 0:
            print(f"Epoch {epoch}/{num_epochs}, Training Loss: {total_loss/num_batches}")
    
    # 테스트 데이터로 최종 성능 평가
    y_pred = sess.run(predicted_value, feed_dict={x: x_test})
    
    # 예측값과 실제값의 차이 합 계산
    difference_sum = sum(abs(y_pred - y_test))
    print(f'\n차이 합: {difference_sum}')

Epoch 0/400, Training Loss: 176.97408126831056
Epoch 50/400, Training Loss: 0.22626867439597845
Epoch 100/400, Training Loss: 0.008084132033400238
Epoch 150/400, Training Loss: 0.003591131520923227
Epoch 200/400, Training Loss: 0.002413956893084105
Epoch 250/400, Training Loss: 0.0016151720596826635
Epoch 300/400, Training Loss: 0.0015860964944586157
Epoch 350/400, Training Loss: 0.0019253810528898612

차이 합: [5.98693113]
